# GraphStack

This notebook creates a simple python stack to create graph structures within the database(using postgresql and psycopg) and to manipulate them using networkx.

In [20]:
import psycopg2

conn = psycopg2.connect(database="reddb", user="postgres", password="postgres", host="127.0.0.1", port="5433")
print("Database connection successful")

cursor = conn.cursor()
print("Got the cursor for this postgres connection")

Database connection successful
Got the cursor for this postgres connection


In [27]:
# create the table for the company hierarchy

cursor.execute('''CREATE TABLE COMPANY (
    id           INT PRIMARY KEY NOT NULL,
    Name         VARCHAR(20) NOT NULL,
    Designation  CHAR(3)     NOT NULL,
    DOJ          DATE        NOT NULL,
    DependId     INT         NOT NULL);''')
print("Table created successfully")

insert into the table, the table holds recursive relations, the root node will have dependId = 0

cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (1, 'Eng1', 'Eng', '1997-09-01', 6)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (2, 'Eng2', 'Eng', '1995-04-25', 6)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (3, 'Eng3', 'Eng', '2003-04-12', 6)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (4, 'Eng4', 'Eng', '2001-05-21', 7)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (5, 'Eng5', 'Eng', '2002-03-16', 7)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (6, 'Pro1', 'PrM', '2005-05-12', 8)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (7, 'Pro2', 'PrM', '2006-09-11', 8)")
cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (8, 'CTO1', 'CTO', '2007-10-02', 0)")
print("inserted into table successfully")

conn.commit()

inserted into table successfully


In [28]:
# try to retrieve from the table

cursor.execute("SELECT * FROM COMPANY")
rows = cursor.fetchall()
for row in rows :
    print(row[0], row[1], row[2], row[3], row[4], "\n")


1 Eng1 Eng 1997-09-01 6 

2 Eng2 Eng 1995-04-25 6 

3 Eng3 Eng 2003-04-12 6 

4 Eng4 Eng 2001-05-21 7 

5 Eng5 Eng 2002-03-16 7 

6 Pro1 PrM 2005-05-12 8 

7 Pro2 PrM 2006-09-11 8 

8 CTO1 CTO 2007-10-02 0 



In [29]:
# perform recursive CTE on the table to get the graph structure starting from the root

cursor.execute('''WITH RECURSIVE traverse AS(
    SELECT id, Name from COMPANY
    WHERE id = 8
    UNION
    SELECT COMPANY.id, COMPANY.Name from COMPANY, traverse
    WHERE COMPANY.DependId = traverse.id)
    SELECT id, Name from traverse;
    ''')

recursive_rows = cursor.fetchall()
for row in recursive_rows :
    print(row[0], row[1], "\n")

# conn.close()

8 CTO1 

6 Pro1 

7 Pro2 

1 Eng1 

2 Eng2 

3 Eng3 

4 Eng4 

5 Eng5 



# NetworkX Graphs

In [30]:
# manipulate the graph using networkx
import networkx as nx
from datetime import date

compGraph = nx.DiGraph()

for row in rows :                           # loop to add nodes to the graph
    compGraph.add_node(row[0], name=row[1], designation=row[2], doj=row[3])
for row in rows :                           # loop to connect nodes in the graph
     if row[4] != 0 :
        compGraph.add_edge(row[0], row[4])

print("The graph in edge representation : ", compGraph.edges())

# print the attributes of nodes, explicitly
print("\nPrinting the attributes of nodes :-")
for node in compGraph.nodes(data=True) :
    print("Name :", node[1]['name'], ", Date of Joining :", node[1]['doj'])

# compute the shortest path between given edges
short_path = nx.shortest_path(compGraph, source=1, target=8)
print("\nThe shortest path from node(1) to node(8) : ", short_path)

# date comparison function to determine if date2 is greater than, less than or equal to date1
def dateCompare(date1, date2) :
    return date2.toordinal() > date1.toordinal()

# get the nodes with joining date less than the current node(=3) value
node_date = compGraph.node[3]['doj']
print("\nThe nodes whose dates ar greater than the current nodes value :- ")
for node in compGraph.nodes(data=True) :
    if dateCompare(node_date, node[1]['doj']) :
        print(node)

# networkx complete

The graph in edge representation :  [(1, 6), (2, 6), (3, 6), (4, 7), (5, 7), (6, 8), (7, 8)]

Printing the attributes of nodes :-
Name : Eng1 , Date of Joining : 1997-09-01
Name : Eng2 , Date of Joining : 1995-04-25
Name : Eng3 , Date of Joining : 2003-04-12
Name : Eng4 , Date of Joining : 2001-05-21
Name : Eng5 , Date of Joining : 2002-03-16
Name : Pro1 , Date of Joining : 2005-05-12
Name : Pro2 , Date of Joining : 2006-09-11
Name : CTO1 , Date of Joining : 2007-10-02

The shortest path from node(1) to node(8) :  [1, 6, 8]

The nodes whose dates ar greater than the current nodes value :- 
(6, {'name': 'Pro1', 'doj': datetime.date(2005, 5, 12), 'designation': 'PrM'})
(7, {'name': 'Pro2', 'doj': datetime.date(2006, 9, 11), 'designation': 'PrM'})
(8, {'name': 'CTO1', 'doj': datetime.date(2007, 10, 2), 'designation': 'CTO'})


In [33]:
# read csv file and create graph in-memory
# import csv
edges = []
csvfile = open('/home/anish/Development/GraphStack/GraphData.csv', 'r')
filereader = csv.reader(csvfile, delimiter=',', quotechar='\'')
for row in filereader :                       # loop to add nodes to graph
    compGraph.add_node(int(row[0]), name=row[1], designation=row[2], doj=date(int(row[3][0:4]), int(row[3][5:7]),
                                                                              int(row[3][8:]) ))
    edges.append((int(row[0]), int(row[4])))

for edge in edges :                           # loop to add edges to graph
    if(compGraph.__contains__(edge[1])) :
        compGraph.add_edge(edge[0], edge[1])


In [47]:
# add the in-memory graph to database

for new_node in edges :
    id = str(new_node[0])
    doj = compGraph.node[new_node[0]]['doj'].isoformat()
    name = compGraph.node[new_node[0]]['name']
    desgn = compGraph.node[new_node[0]]['designation']
    depenId = str(new_node[1])
    value_str = "%s, '%s', '%s', '%s', %s" %(id, name, desgn, doj, depenId)

    cursor.execute("INSERT INTO COMPANY(id, Name, Designation, DOJ, DependId) VALUES (" + value_str + ")")

print("Saved in-memory graph to database")

# check the new addititons by performing recursive CTE

print("\nRecursive traversal of the graph after inserting data :-")
cursor.execute('''WITH RECURSIVE traverse AS(
    SELECT id, Name from COMPANY
    WHERE id = 8
    UNION
    SELECT COMPANY.id, COMPANY.Name from COMPANY, traverse
    WHERE COMPANY.DependId = traverse.id)
    SELECT id, Name from traverse;
    ''')

recursive_rows = cursor.fetchall()
for row in recursive_rows :
    print(row[0], row[1], "\n")

# conn.close()

Saved in-memory graph to database

Recursive traversal of the graph after inserting data :-
8 CTO1 

6 Pro1 

7 Pro2 

15 Pro3 

16 Pro4 

1 Eng1 

2 Eng2 

3 Eng3 

4 Eng4 

5 Eng5 

9 Eng6 

10 Eng7 

11 Eng8 

12 Eng9 

13 Eng10 

14 Eng11 



In [48]:
import matplotlib.pyplot as plt

# draw the graph
nx.draw_networkx(compGraph, node_color="blue", edge_color="blue", font_color="white")
plt.show()

In [49]:
# flask-task